LOADING THE DATA

In [ ]:
data = read.table("http://www-stat.stanford.edu/~tibs/ElemStatLearn/datasets/LAozone.data",sep=",",head=T)

PACKAGES

In [ ]:
install.packages("psych")
install.packages("reshape2")
install.packages("caret")
install.packages("caTools")
install.packages("olsrr")
install.packages("car")
install.packages("glmnet")

#package installation

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘mnormt’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘plyr’, ‘Rcpp’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘pROC’, ‘recipes’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘car’, ‘g

In [ ]:
X <- c("vh", "wind", "humidity", "temp", "ibh", "dpg", "ibt", "vis")
target <- data$ozone
data$doy <- NULL

#predictor list and response variable defined
#note that "doy" predictor is omitted as it merely serves as identification for each observation

EXPLORING THE DATA

In [ ]:
anyNA(X) #checking for missing values across all eight predictors. If the function returns "FALSE" then there are none:)

[1] FALSE

In [ ]:
library(psych) #getting summary stats for each of the predictors
summary <- describe(data[, X])
summary

In [ ]:
model_all_preds <- lm(target ~ temp + wind + ibh + ibt + vis + humidity + dpg, data = data)

cooks_distance <- cooks.distance(model_all_preds)
cook_threshold <- 4 / nrow(data)

influential_points <- which(cooks_distance > cook_threshold)
data <- data[-influential_points, ]

#using cooks distance to identify and remove influential points

In [ ]:
X <- data[, c("vh", "wind", "humidity", "temp", "ibh", "dpg", "ibt", "vis")]
target <- data$ozone

#redefining predictor list as a vector for computation and regression

Here, I made a multiple linear regression model with all of the predictors for the exploration of influential points using cook's distance. The influential points are now removed.

In [ ]:
par(mfrow = c(4,2))
for (col_name in colnames(X)) {
    predictor_data <- X[[col_name]]
    plot(predictor_data, target, pch = 19, main = paste("Scatter Plot for Predictor", col_name), xlab = col_name, ylab = "Target")
    hist(predictor_data, breaks = 20, col = "blue", main = paste("Histogram for Predictor", col_name), xlab = col_name)
}
#descriptive graphs generated through a for-loop
#includes histograms, scatterplots and box plots

In [ ]:
model_mult = lm(target ~ vh + temp + wind + humidity + ibh + ibt + dpg + vis, data = data)

rsquared_mult <- summary(model_mult)$r.squared
cat("R-squared:", rsquared, "\n")

predictions_mult <- predict(model_mult, newdata = data)

mse_mult <- mean((data$target - predictions_mult)^2)
cat("Mean Squared Error:", mse, "\n")

#calculating R2 and MSE for multiple regression model

In [ ]:
plot(model_mult, which = 1, main = "Residuals vs Fitted")

plot(model_mult, which = 2, main = "Normal Q-Q")

#Residual and QQ plot for multiple regression model

In [ ]:
library(car)

vif_values <- vif(model_mult)

print(vif_values)

Significant multicollinearity is present in variables exhibiting a VIF close to or greater than 10. Thus, temp and ibt are the two most problematic predictors. We will remove them from the data

VERIFYING LINEAR REGRESSION ASSUMPTIONS

1)EXISTENCE: looking at the scatter plots and summary stats of each predictor, this condition is satisfied

2)INDEPENDENCE: each observation has only one recorded value for each predictor, so we can assume the Y values are independent.

3)LINEARITY: seeing as though most of the residual plots are curvilinear, linearity has been violated. We attempted box-cox and square transformation
on the response variable, but this did not rectify the curvilinear patterns in the residual plots.

4)HOMOSCEDASTICITY: looking at the residual plots for each predictor, all eight predictors exhibit heteroscedasticity. That being said, this
assumption has also been violated. However, for the purpose of this project, we will continue with this condition present as there is no
appropriate way to fix the severity of the heterscedasticity.

5)NORMALITY: Looking at the eight QQ plots, our predictors are all normal, so this assumption is strongly fulfilled.

In summary, linearity and homoscedasticity are both significantly violated. However, Applying two different transformations to the target did not quill these violations, so there is nothing further we can do at this time.

EXPLORING COLLINEARITY

In [ ]:
library(reshape2)
library(ggplot2)
X <- c("vh", "wind", "humidity", "temp", "ibh", "dpg", "ibt", "vis")
correlation_matrix <- cor(data[, X])
correlation_matrix

ggplot2::ggplot(data = melt(correlation_matrix), aes(Var1, Var2, fill = value)) +
  geom_tile() +
  scale_fill_gradient2() +
  theme_minimal()

Here, we see that there is some collinearity going on. While there are many methods to remedy collinearity, the best option for this data is to remove one of these predictors and check for collinearity again, repeating the process until the heatmap displays little to no collinearity. We examine the relationships between vh and temp with a correlation of .81, vh and ibt with a correlation of .85, temp and ibt with a correlation of .86, and ibt and ibh with a correlation of -.77. First, we will remove ibt since is displays collinearity with two other predictors and has the highest VIF value

In [ ]:
library(reshape2)

drop_ibt <- c("wind","humidity","vh","ibh","dpg","temp","vis")
correlation_matrix <- cor(data[, drop_ibt])
correlation_matrix

library(ggplot2)
ggplot2::ggplot(data = melt(correlation_matrix), aes(Var1, Var2, fill = value)) + geom_tile() + scale_fill_gradient2() + theme_minimal()

The heat map displays less collinearity without ibt. Although temp has a VIF of 8.9, we will not remove it since research shows that temperature is highly correlated with ozone and in that case should not be removed from the model. We've now addressed multicollinearity in our model.

Now that collinearity has been mitigated, let's move on to finding the best model for our data. The method chosen here is backward selection since the model displayed collinearity before two predictors were eliminated. Backward selection helps to address multicollinearity. Thus, using backward selection will keep the current model we have OR remove another predictor.

BACKWARD SELECTION

In [ ]:
library(olsrr)
model <- lm(
  formula = target ~ temp + wind	+ humidity + vh	+ ibh	+ dpg	+ vis,
  data = data
)

BWDfit.p <- ols_step_backward_p(model, prem = 0.05)
BWDfit.p
backward_model <- ols_step_backward_p(model, prem = 0.05, details = TRUE)

Multicollinearity can make it difficult to separate the individual effects of predictor variables, including interaction terms. It can inflate the standard errors of the coefficients, making them less precise and leading to a lack of statistical significance. Thus, we will not attempt checking for/including interaction terms. As we progress in our model building process, we will now conduct stepwise regression.

STEPWISE REGRESSION

In [ ]:
model <- lm(target ~ temp + wind	+ humidity + vh	+ ibh	+ dpg	+ vis, data = data)
step_model <- step(model, direction = "backward", k = 2, trace = 2)
summary(step_model)

predictions_step <- predict(step_model, newdata = data)
rsquared_step <- 1 - sum((target - predictions_step)^2) / sum((target - mean(target))^2)
mse_step <- mean((target - predictions_step)^2)

cat("R-squared:", rsquared_step, "\n")
cat("Mean Squared Error (MSE):", mse_step, "\n")

The result of our stepwise regression is a model including humidity, temp, ibh, and vis. Compared to the model found through backward selection, this model includes one less predictor (vis).Additionally, the R-square of this model is .6869 while the R-square of the backward selection model is .602. The backward selection model includes five predictors but this stepwise model only includes four, which allows for better model interpretability. Thus, we prefer the stepwise at this point. Let me note at this point that since multicollinearity was present before eliminating the collinear variables, performing ridge regression is appropriate since its main goal is to rectify collinearity. However, ridge regression only shrinks coefficients down towards zero rather than performing variable selection. The model will have poor interpretability, but we will still perform it to observe how it addresses multicollinearity.

RIDGE REGRESSION

In [ ]:
library(glmnet)

x <- data.matrix(data[, c('temp', 'vis', 'ibh', 'ibt', 'wind','vh','humidity','dpg')])
model <- glmnet(x, target, alpha = 0)

cv_model <- cv.glmnet(x, target, alpha = 0)
best_lambda <- cv_model$lambda.min
plot(cv_model)

ridge_model <- glmnet(x, target, alpha = 0, lambda = best_lambda)
coef(ridge_model)

In [ ]:
predictions <- predict(ridge_model, newx = x, s = best_lambda)
rsquared_ridge <- 1 - sum((target - predictions)^2) / sum((target - mean(target))^2)
mse_ridge <- mean((target - predictions)^2)

cat("R-squared:", rsquared_ridge, "\n")
cat("Mean Squared Error (MSE):", mse_ridge, "\n")

LASSO REGRESSION

In [ ]:
cv_model <- cv.glmnet(x, target, alpha = 1)
best_lambda <- cv_model$lambda.min
plot(cv_model)

lasso_model <- glmnet(x, target, alpha = 1, lambda = best_lambda)
coef(lasso_model)

predictions_lasso <- predict(lasso_model, newx = x, s = best_lambda)
rsquared_lasso <- 1 - sum((target - predictions_lasso)^2) / sum((target - mean(target))^2)
mse_lasso <- mean((target - predictions_lasso)^2)
print(paste("R-squared:", rsquared_lasso))
print(paste("Mean Squared Error:", mse_lasso))

In [ ]:
multiple_reg <- lm(target ~ vh + temp + wind + ibh + ibt + vis + humidity + dpg, data = data)

summary(multiple_reg)

predictions_mult <- predict(multiple_reg, newdata = data)
rsquared_mult <- 1 - sum((target - predictions_mult)^2) / sum((target - mean(target))^2)
mse_mult <- mean((target - predictions_mult)^2)

cat("R-squared:", rsquared_mult, "\n")
cat("Mean Squared Error (MSE):", mse_mult, "\n")


Below is what I'm using for variable significance testing for Backwards selection

In [ ]:
multiple_reg <- lm(target ~ temp + ibh + humidity, data = data)

summary(multiple_reg)

predictions_mult <- predict(multiple_reg, newdata = data)
rsquared_mult <- 1 - sum((target - predictions_mult)^2) / sum((target - mean(target))^2)
mse_mult <- mean((target - predictions_mult)^2)

cat("R-squared:", rsquared_mult, "\n")
cat("Mean Squared Error (MSE):", mse_mult, "\n")

Below is what I'm using for variabler signficance testing for Stepwise Selection

In [ ]:
multiple_reg <- lm(target ~ temp + ibh+ vis + humidity, data = data)

summary(multiple_reg)

predictions_mult <- predict(multiple_reg, newdata = data)
rsquared_mult <- 1 - sum((target - predictions_mult)^2) / sum((target - mean(target))^2)
mse_mult <- mean((target - predictions_mult)^2)

cat("R-squared:", rsquared_mult, "\n")
cat("Mean Squared Error (MSE):", mse_mult, "\n")

Below is what I'm using for Lasso Regression variable significance testing

In [ ]:
multiple_reg <- lm(target ~ temp + ibh + ibt + vis + humidity, data = data)

summary(multiple_reg)

predictions_mult <- predict(multiple_reg, newdata = data)
rsquared_mult <- 1 - sum((target - predictions_mult)^2) / sum((target - mean(target))^2)
mse_mult <- mean((target - predictions_mult)^2)

cat("R-squared:", rsquared_mult, "\n")
cat("Mean Squared Error (MSE):", mse_mult, "\n")